In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "_scripts"))

In [15]:
from case_edits.ezcase_new import *
from helpers.ep_helpers import *
from helpers.helpers import *
from plan.helpers import get_plans_from_file
from new_subsurfaces.creator import create_partner_name, PARTNER, reverse_partner_name

In [4]:

inputs_dir = "case_red_d3"
outputs_dir = "tests/test22_svg2plan"

case = initialize_case(outputs_dir)
path_to_input = get_path_to_inputs(inputs_dir)
pairs = get_subsurface_pairs_from_case(path_to_input)




In [5]:
case = create_ezcase(outputs_dir, inputs_dir)
idf = case.idf

out.idf exists
IDF has changed: True


In [15]:
plans = get_plans_from_file(path_to_input)
[i.get_node_data() for i in plans]


[(0, 'kitchen', Coord(x=0.0, y=3.15)),
 (1, 'living', Coord(x=0.0, y=1.78)),
 (2, 'transit', Coord(x=1.7, y=1.8)),
 (3, 'bedroom_2', Coord(x=2.54, y=1.23)),
 (4, 'bedroom', Coord(x=2.36, y=3.15)),
 (5, 'closet', Coord(x=3.25, y=1.8)),
 (6, 'transit_2', Coord(x=2.54, y=1.8)),
 (7, 'bath', Coord(x=1.7, y=3.15))]

In [11]:
def find_matching_subsurface(zone_name: str, subsurfaces: list[EpBunch]):
    return [s.Name for s in subsurfaces if zone_name in s.Building_Surface_Name]
        

In [19]:
zones = get_zones(idf)
subsurfaces = idf.getsubsurfaces()
zone_map = {z.Name: find_matching_subsurface(z.Name, subsurfaces) for z in zones}
zone_map

{'Block 00 Storey 0': ['Block 01 Storey 0 Wall 0002_1 Door Partner',
  'Block 00 Storey 0 Wall 0003 Window',
  'Block 00 Storey 0 Wall 0002 Window'],
 'Block 01 Storey 0': ['Block 01 Storey 0 Wall 0001_1 Door',
  'Block 01 Storey 0 Wall 0002_1 Door',
  'Block 01 Storey 0 Wall 0003 Window',
  'Block 01 Storey 0 Wall 0004 Window'],
 'Block 02 Storey 0': ['Block 01 Storey 0 Wall 0001_1 Door Partner',
  'Block 02 Storey 0 Wall 0001_2 Door',
  'Block 02 Storey 0 Wall 0001_1 Door',
  'Block 02 Storey 0 Wall 0002_2 Door'],
 'Block 03 Storey 0': ['Block 02 Storey 0 Wall 0001_1 Door Partner',
  'Block 03 Storey 0 Wall 0002_2 Door',
  'Block 03 Storey 0 Wall 0004 Window'],
 'Block 04 Storey 0': ['Block 06 Storey 0 Wall 0002_1 Door Partner',
  'Block 04 Storey 0 Wall 0001 Window',
  'Block 04 Storey 0 Wall 0002 Window'],
 'Block 05 Storey 0': [],
 'Block 06 Storey 0': ['Block 02 Storey 0 Wall 0001_2 Door Partner',
  'Block 06 Storey 0 Wall 0002_1 Door',
  'Block 03 Storey 0 Wall 0002_2 Door Partn

In [21]:
for k, v in zone_map.items():
    for ix, name in enumerate(v):
        if PARTNER in name:
            zone_map[k][ix] = reverse_partner_name(name)

In [22]:
zone_map

{'Block 00 Storey 0': ['Block 01 Storey 0 Wall 0002_1 Door',
  'Block 00 Storey 0 Wall 0003 Window',
  'Block 00 Storey 0 Wall 0002 Window'],
 'Block 01 Storey 0': ['Block 01 Storey 0 Wall 0001_1 Door',
  'Block 01 Storey 0 Wall 0002_1 Door',
  'Block 01 Storey 0 Wall 0003 Window',
  'Block 01 Storey 0 Wall 0004 Window'],
 'Block 02 Storey 0': ['Block 01 Storey 0 Wall 0001_1 Door',
  'Block 02 Storey 0 Wall 0001_2 Door',
  'Block 02 Storey 0 Wall 0001_1 Door',
  'Block 02 Storey 0 Wall 0002_2 Door'],
 'Block 03 Storey 0': ['Block 02 Storey 0 Wall 0001_1 Door',
  'Block 03 Storey 0 Wall 0002_2 Door',
  'Block 03 Storey 0 Wall 0004 Window'],
 'Block 04 Storey 0': ['Block 06 Storey 0 Wall 0002_1 Door',
  'Block 04 Storey 0 Wall 0001 Window',
  'Block 04 Storey 0 Wall 0002 Window'],
 'Block 05 Storey 0': [],
 'Block 06 Storey 0': ['Block 02 Storey 0 Wall 0001_2 Door',
  'Block 06 Storey 0 Wall 0002_1 Door',
  'Block 03 Storey 0 Wall 0002_2 Door'],
 'Block 07 Storey 0': ['Block 02 Storey 0 